In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import mannwhitneyu, anderson
from statsmodels.stats.multitest import multipletests
from sklearn.utils import resample

# Константы для анализа
ALPHA = 0.05
N_BOOTSTRAPS = 5000

# ==============================================
# 1. ЗАГРУЗКА ДАННЫХ
# ==============================================

# Загрузка данных
df = pd.read_excel('hw_1_ds.xlsx')

# Добавляем расчетные столбцы ДО фильтрации
df['date'] = df['datetime'].dt.date
df['is_test'] = ~df['control_group_flg']
df['bonus_pct'] = np.where(
    df['monetary_amt'] > 0,
    df['bonus_turn_amt'] / df['monetary_amt'] * 100,
    0
)

# Удаляем нулевые транзакции и дубликаты (если нужна строгая уникальность)
df_clean = (
    df[df['transaction_amt'] > 0]
    .drop_duplicates(subset=['customer_id', 'transaction_id'])  # На случай дублей транзакций
    .copy()
)

# ==============================================
# 2. ПОМЕТКА ПЕРВЫХ ПОКУПОК
# ==============================================

# Добавляем ранжирование транзакций по времени для каждого клиента
df_analysis = df_clean.copy()

# Сортировка по клиенту и времени
df_analysis = df_analysis.sort_values(['customer_id', 'datetime'])

# Помечаем первую транзакцию для каждого клиента
df_analysis['transaction_rank'] = df_analysis.groupby('customer_id').cumcount() + 1
df_analysis['is_first_transaction'] = df_analysis['transaction_rank'] == 1

# ==============================================
# 3. ПРОВЕРКА ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ МЕЖДУ ГРУППАМИ
# ==============================================

print("=== АНАЛИЗ ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ ===\n")

# Разделение по акциям
df_7_new = df_analysis[df_analysis['promo'].str.contains('7% на Новых', na=False)].copy()
df_6_otk = df_analysis[df_analysis['promo'].str.contains('6% на Отток', na=False)].copy()

# Выделение групп для акции 7%
test_7_customers = set(df_7_new[~df_7_new['control_group_flg']]['customer_id'].unique())
control_7_customers = set(df_7_new[df_7_new['control_group_flg']]['customer_id'].unique())

# Выделение групп для акции 6%
test_6_customers = set(df_6_otk[~df_6_otk['control_group_flg']]['customer_id'].unique())
control_6_customers = set(df_6_otk[df_6_otk['control_group_flg']]['customer_id'].unique())

# Проверка пересечений для акции 7%
intersection_7_test_control = test_7_customers.intersection(control_7_customers)
intersection_7_test_6 = test_7_customers.intersection(test_6_customers)
intersection_7_control_6_control = control_7_customers.intersection(control_6_customers)

# Проверка пересечений для акции 6%
intersection_6_test_control = test_6_customers.intersection(control_6_customers)

print(f"Акция 7% Новые:")
print(f"  Тестовая группа: {len(test_7_customers)} клиентов")
print(f"  Контрольная группа: {len(control_7_customers)} клиентов")
print(f"  Пересечение тест/контроль: {len(intersection_7_test_control)} клиентов")

print(f"\nАкция 6% Отток:")
print(f"  Тестовая группа: {len(test_6_customers)} клиентов")
print(f"  Контрольная группа: {len(control_6_customers)} клиентов")
print(f"  Пересечение тест/контроль: {len(intersection_6_test_control)} клиентов")

print(f"\nМежакционные пересечения:")
print(f"  Тест 7% ∩ Тест 6%: {len(intersection_7_test_6)} клиентов")
print(f"  Контроль 7% ∩ Контроль 6%: {len(intersection_7_control_6_control)} клиентов")

# Флаг критической проблемы
if len(intersection_7_test_control) > 0 or len(intersection_6_test_control) > 0:
    print(f"\nЕсть клиенты в обеих группах одной акции!")

# ==============================================
# 4. ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ
# ==============================================
print("")
print("=== ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ ===\n")

# Функция для детального анализа группы
def analyze_group_transactions(df_group, group_name):
    total_transactions = len(df_group)
    first_transactions = df_group['is_first_transaction'].sum()
    repeat_transactions = (~df_group['is_first_transaction']).sum()
    
    repeat_pct = (repeat_transactions / total_transactions * 100) if total_transactions > 0 else 0
    
    unique_customers = df_group['customer_id'].nunique()
    
    print(f"{group_name}:")
    print(f"  Уникальных клиентов: {unique_customers}")
    print(f"  Всего транзакций: {total_transactions}")
    print(f"  Первые транзакции: {first_transactions}")
    print(f"  Повторные транзакции: {repeat_transactions} ({repeat_pct:.1f}%)")
    
    return {
        'customers': unique_customers,
        'total_transactions': total_transactions,
        'first_transactions': first_transactions,
        'repeat_transactions': repeat_transactions,
        'repeat_pct': repeat_pct
    }

# Анализ для акции 7% Новые
df_7_new_test = df_7_new[~df_7_new['control_group_flg']]
df_7_new_control = df_7_new[df_7_new['control_group_flg']]

print("Акция 7% Новые:")
stats_7_test = analyze_group_transactions(df_7_new_test, "  Тестовая группа")
stats_7_control = analyze_group_transactions(df_7_new_control, "  Контрольная группа")

# Анализ для акции 6% Отток
df_6_otk_test = df_6_otk[~df_6_otk['control_group_flg']]
df_6_otk_control = df_6_otk[df_6_otk['control_group_flg']]

print("\nАкция 6% Отток:")
stats_6_test = analyze_group_transactions(df_6_otk_test, "  Тестовая группа")
stats_6_control = analyze_group_transactions(df_6_otk_control, "  Контрольная группа")

# Проверка баланса групп
print("\n=== АНАЛИЗ БАЛАНСА ГРУПП ===")
balance_7 = abs(stats_7_test['customers'] - stats_7_control['customers']) / (stats_7_test['customers'] + stats_7_control['customers']) * 100
balance_6 = abs(stats_6_test['customers'] - stats_6_control['customers']) / (stats_6_test['customers'] + stats_6_control['customers']) * 100

print(f"Дисбаланс групп 7% Новые: {balance_7:.1f}%")
print(f"Дисбаланс групп 6% Отток: {balance_6:.1f}%")

=== АНАЛИЗ ПЕРЕСЕЧЕНИЙ КЛИЕНТОВ ===

Акция 7% Новые:
  Тестовая группа: 13378 клиентов
  Контрольная группа: 1474 клиентов
  Пересечение тест/контроль: 0 клиентов

Акция 6% Отток:
  Тестовая группа: 3545 клиентов
  Контрольная группа: 342 клиентов
  Пересечение тест/контроль: 0 клиентов

Межакционные пересечения:
  Тест 7% ∩ Тест 6%: 0 клиентов
  Контроль 7% ∩ Контроль 6%: 0 клиентов

=== ДЕТАЛИЗИРОВАННЫЙ АНАЛИЗ ПО ГРУППАМ ===

Акция 7% Новые:
  Тестовая группа:
  Уникальных клиентов: 13378
  Всего транзакций: 16295
  Первые транзакции: 13378
  Повторные транзакции: 2917 (17.9%)
  Контрольная группа:
  Уникальных клиентов: 1474
  Всего транзакций: 1775
  Первые транзакции: 1474
  Повторные транзакции: 301 (17.0%)

Акция 6% Отток:
  Тестовая группа:
  Уникальных клиентов: 3545
  Всего транзакций: 4622
  Первые транзакции: 3545
  Повторные транзакции: 1077 (23.3%)
  Контрольная группа:
  Уникальных клиентов: 342
  Всего транзакций: 433
  Первые транзакции: 342
  Повторные транзакции: 91 

## 📊 ВЫВОДЫ ПО АНАЛИЗУ ДАННЫХ

### 📈 **СТРУКТУРА ВЫБОРОК:**
- Акция 7% Новые: 13,378 клиентов в тесте vs 1,474 в контроле (дисбаланс 80.2%)
- Акция 6% Отток: 3,545 клиентов в тесте vs 342 в контроле (дисбаланс 82.4%)
- **Дисбаланс групп превышает 80% в обеих акциях, что требует особого внимания при интерпретации результатов**

### 📊 **ПОВЕДЕНЧЕСКИЕ ПАТЕРНЫ:**
- Уровень повторных транзакций в тесте и контроле практически идентичен:
  - 7% Новые: 17.9% vs 17.0%
  - 6% Отток: 23.3% vs 21.0%
- Группа "Отток" демонстрирует более высокую склонность к повторным покупкам по сравнению с "Новыми"

In [2]:
# ==============================================
# 5. РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА (БЕЗ ДУБЛИРОВАНИЯ)
# ==============================================
print("=== РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА ===\n")

def calculate_client_metrics(df_group, group_name):
    """Расчет метрик на уровне клиента для заданной группы"""
    
    client_metrics = df_group.groupby('customer_id').apply(
        lambda x: pd.Series({
            'avg_check': x['transaction_amt'].mean(),
            'transaction_count': len(x),
            'has_repeat_purchase': int(len(x) > 1),
            'total_bonus': x['bonus_turn_amt'].sum(),
            'total_revenue': x['monetary_amt'].sum(),
            'additional_cost': (
                (x[x['is_first_transaction']]['bonus_pct'] - 3.0) * 
                x[x['is_first_transaction']]['monetary_amt']
            ).sum() if x['is_first_transaction'].any() else 0,
            # Добавляем метрику repeat_purchase_rate для лучшего понимания
        })
    ).reset_index()
    
    print(f"{group_name}:")
    print(f"  Клиентов: {len(client_metrics)}")
    print(f"  Средний чек: {client_metrics['avg_check'].mean():.2f}")
    print(f"  Среднее кол-во транзакций: {client_metrics['transaction_count'].mean():.2f}")
    print(f"  Доля повторных покупок: {client_metrics['has_repeat_purchase'].mean()*100:.1f}%")
    print(f"  Средние доп. расходы: {client_metrics['additional_cost'].mean():.2f}")
    
    return client_metrics

# Расчет метрик для всех групп (ОДИН РАЗ)
print("Акция 7% Новые:")
client_metrics_7_test = calculate_client_metrics(df_7_new_test, "  Тестовая группа")
client_metrics_7_control = calculate_client_metrics(df_7_new_control, "  Контрольная группа")

print("\nАкция 6% Отток:")
client_metrics_6_test = calculate_client_metrics(df_6_otk_test, "  Тестовая группа")
client_metrics_6_control = calculate_client_metrics(df_6_otk_control, "  Контрольная группа")

# Сохранение метрик для дальнейшего анализа
metrics_storage = {
    '7_new_test': client_metrics_7_test,
    '7_new_control': client_metrics_7_control,
    '6_otk_test': client_metrics_6_test,
    '6_otk_control': client_metrics_6_control
}

print(f"\n✅ Метрики рассчитаны для всех групп")
print(f"📊 Данные готовы для статистического анализа")

=== РАСЧЕТ МЕТРИК НА УРОВНЕ КЛИЕНТА ===

Акция 7% Новые:
  Тестовая группа:
  Клиентов: 13378
  Средний чек: 664.16
  Среднее кол-во транзакций: 1.22
  Доля повторных покупок: 16.5%
  Средние доп. расходы: 1843.48
  Контрольная группа:
  Клиентов: 1474
  Средний чек: 623.12
  Среднее кол-во транзакций: 1.20
  Доля повторных покупок: 15.4%
  Средние доп. расходы: 0.05

Акция 6% Отток:
  Тестовая группа:
  Клиентов: 3545
  Средний чек: 689.55
  Среднее кол-во транзакций: 1.30
  Доля повторных покупок: 21.8%
  Средние доп. расходы: 1467.73
  Контрольная группа:
  Клиентов: 342
  Средний чек: 652.56
  Среднее кол-во транзакций: 1.27
  Доля повторных покупок: 20.2%
  Средние доп. расходы: 0.06

✅ Метрики рассчитаны для всех групп
📊 Данные готовы для статистического анализа


In [3]:
# ==============================================
# 6. РАСЧЕТ МОЩНОСТИ ТЕСТА (после статистического анализа)
# ==============================================

def calculate_power(effect_size, n, alpha=ALPHA):
    """Расчет мощности двухвыборочного t-теста."""
    from statsmodels.stats.power import ttest_power
    try:
        # abs() для effect_size, так как направление эффекта не влияет на мощность
        power = ttest_power(effect_size=abs(effect_size), nobs=n, alpha=alpha, alternative='two-sided')
        return power
    except Exception as e:
        # Если расчет невозможен (например, effect_size=0 или NaN)
        print(f"  Предупреждение: Невозможно рассчитать мощность: {e}")
        return np.nan

# Анализ мощности для ключевых метрик на основе результатов бутстрэпа
# Используем данные из corrected_ab_results, где уже есть sample_size и observed_diff
# Для расчета Cohen's d нам нужны стандартные отклонения групп
# Пересчитаем их для получения более точной оценки мощности

power_analysis_summary = {}

for promo_name, test_data, control_data in [
    ('7%_new', client_metrics_7_test, client_metrics_7_control),
    ('6%_otk', client_metrics_6_test, client_metrics_6_control)
]:
    equal_size = min(len(test_data), len(control_data))
    print(f"--- АКЦИЯ {promo_name} (размер групп: {equal_size}) ---")
    power_analysis_summary[promo_name] = {}
    
    # Определим метрики для анализа мощности
    # Основной фокус - на поведенческих метриках. additional_cost и total_bonus заведомо покажут высокую мощность,
    # так как акция прямо на них влияет.
    metrics_for_power = ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue']
    
    for metric in metrics_for_power:
        # Получаем данные для расчета мощности
        test_metric_data = test_data[metric].dropna()
        control_metric_data = control_data[metric].dropna()
        
        # Повторяем расчет разницы средних (для согласованности)
        mean_test = test_metric_data.mean()
        mean_control = control_metric_data.mean()
        observed_diff = mean_test - mean_control
        
        # Рассчитываем стандартные отклонения
        std_test = test_metric_data.std()
        std_control = control_metric_data.std()
        
        # Рассчитываем pooled standard deviation (для Cohen's d)
        # pooled_std = sqrt(((n1-1)*s1^2 + (n2-1)*s2^2) / (n1+n2-2))
        # При равных размерах групп (n1=n2=n): pooled_std = sqrt((s1^2 + s2^2)/2)
        if equal_size > 1 and std_test >= 0 and std_control >= 0 and not (std_test == 0 and std_control == 0):
            pooled_std = np.sqrt((std_test**2 + std_control**2) / 2)
            
            # Рассчитываем Cohen's d
            if pooled_std > 0:
                cohens_d = observed_diff / pooled_std
            else:
                cohens_d = np.nan # Невозможно рассчитать, если нет вариации
                
            # Рассчитываем мощность
            power = calculate_power(cohens_d, equal_size)
            
            power_analysis_summary[promo_name][metric] = {
                'effect_size_d': cohens_d,
                'sample_size': equal_size,
                'power': power,
                'mean_test': mean_test,
                'mean_control': mean_control,
                'std_test': std_test,
                'std_control': std_control
            }
            
            print(f"{metric:25} | d={cohens_d:6.3f} | Мощность={power:5.3f} | Diff={observed_diff:8.2f}")
            
            if not np.isnan(power) and power < 0.8:
                print(f"       ⚠️  ВНИМАНИЕ: Мощность ({power:.1%}) < 80%. Риск пропустить реальный эффект высок.")
        else:
            print(f"{metric:25} | Недостаточно данных или нулевая дисперсия для расчета мощности.")

--- АКЦИЯ 7%_new (размер групп: 1474) ---
avg_check                 | d= 0.045 | Мощность=0.407 | Diff=   41.05
       ⚠️  ВНИМАНИЕ: Мощность (40.7%) < 80%. Риск пропустить реальный эффект высок.
transaction_count         | d= 0.025 | Мощность=0.156 | Diff=    0.01
       ⚠️  ВНИМАНИЕ: Мощность (15.6%) < 80%. Риск пропустить реальный эффект высок.
has_repeat_purchase       | d= 0.029 | Мощность=0.203 | Diff=    0.01
       ⚠️  ВНИМАНИЕ: Мощность (20.3%) < 80%. Риск пропустить реальный эффект высок.
total_revenue             | d= 0.046 | Мощность=0.418 | Diff=   40.00
       ⚠️  ВНИМАНИЕ: Мощность (41.8%) < 80%. Риск пропустить реальный эффект высок.
--- АКЦИЯ 6%_otk (размер групп: 342) ---
avg_check                 | d= 0.029 | Мощность=0.082 | Diff=   36.99
       ⚠️  ВНИМАНИЕ: Мощность (8.2%) < 80%. Риск пропустить реальный эффект высок.
transaction_count         | d= 0.056 | Мощность=0.178 | Diff=    0.04
       ⚠️  ВНИМАНИЕ: Мощность (17.8%) < 80%. Риск пропустить реальный эффект в

In [4]:
# ==============================================
# 7. СТАТИСТИЧЕСКИЙ АНАЛИЗ (КВАЗИ-ЭКСПЕРИМЕНТ)
# ==============================================

def bootstrap_balanced_ab_test(test_data, control_data, metric, n_iterations=N_BOOTSTRAPS, sample_size=None, alpha=ALPHA):
    """
    Проводит бутстрэп-анализ для сравнения средних двух групп с балансировкой размеров выборок.

    Аргументы:
        test_data (pd.DataFrame): Данные тестовой группы.
        control_data (pd.DataFrame): Данные контрольной группы.
        metric (str): Название метрики для анализа.
        n_iterations (int): Количество итераций бутстрэпа.
        sample_size (int, optional): Размер выборки для каждой группы после балансировки.
                                     Если None, берется минимум из размеров групп.
        alpha (float): Уровень значимости (по умолчанию 0.05).

    Возвращает:
        dict: Словарь с результатами анализа.
    """
    test_metric = test_data[metric].dropna()
    control_metric = control_data[metric].dropna()

    if sample_size is None:
        sample_size = min(len(test_metric), len(control_metric))
    
    # Наблюдаемая разница на полных данных
    observed_diff = test_metric.mean() - control_metric.mean()
    
    # Бутстрэп на сбалансированных выборках
    bootstrap_diffs = []
    for _ in range(n_iterations):
        # Случайная выборка равных размеров
        boot_test = resample(test_metric, n_samples=sample_size, random_state=None)
        boot_control = resample(control_metric, n_samples=sample_size, random_state=None)
        
        diff = boot_test.mean() - boot_control.mean()
        bootstrap_diffs.append(diff)
    
    # Статистики
    ci_lower = np.percentile(bootstrap_diffs, 100 * (alpha/2))  # 2.5%
    ci_upper = np.percentile(bootstrap_diffs, 100 * (1 - alpha/2)) # 97.5%
    
    n_extreme = np.sum(np.abs(bootstrap_diffs) >= np.abs(observed_diff))
    p_value = n_extreme / n_iterations
    
    is_significant = p_value < alpha
    
    return {
        'observed_diff': observed_diff,
        'ci_lower': ci_lower,
        'ci_upper': ci_upper,
        'p_value': p_value,
        'significant': is_significant,
        'sample_size': sample_size,
        'n_bootstrap': n_iterations
    }

# Применение корректного метода
print("=== СТАТИСТИЧЕСКИЙ АНАЛИЗ (КВАЗИ-ЭКСПЕРИМЕНТ) С БАЛАНСИРОВКОЙ ===\n")

# Анализ с балансировкой для обеих акций
corrected_ab_results = {}

for promo_name, test_data, control_data in [
    ('7%_new', client_metrics_7_test, client_metrics_7_control),
    ('6%_otk', client_metrics_6_test, client_metrics_6_control)
]:
    equal_size = min(len(test_data), len(control_data))
    print(f"--- АКЦИЯ {promo_name} (размер групп после балансировки: {equal_size}) ---")
    
    corrected_ab_results[promo_name] = {}
    
    # Определим метрики для анализа
    metrics_to_analyze = ['avg_check', 'transaction_count', 'has_repeat_purchase', 'total_revenue', 'total_bonus', 'additional_cost']
    
    for metric in metrics_to_analyze:
        result = bootstrap_balanced_ab_test(test_data, control_data, metric, sample_size=equal_size)
        corrected_ab_results[promo_name][metric] = result
        
        significance_marker = "⭐" if result['significant'] else ""
        print(f"{metric:20} | Diff: {result['observed_diff']:8.2f} | "
              f"95% CI: [{result['ci_lower']:7.2f}, {result['ci_upper']:7.2f}] | "
              f"p-value = {result['p_value']:.3f} {significance_marker}")
    
    print()

print("✅ Статистический анализ (квази-эксперимент) с балансировкой завершен")

=== СТАТИСТИЧЕСКИЙ АНАЛИЗ (КВАЗИ-ЭКСПЕРИМЕНТ) С БАЛАНСИРОВКОЙ ===

--- АКЦИЯ 7%_new (размер групп после балансировки: 1474) ---
avg_check            | Diff:    41.05 | 95% CI: [ -24.29,  106.65] | p-value = 0.516 
transaction_count    | Diff:     0.01 | 95% CI: [  -0.03,    0.05] | p-value = 0.587 
has_repeat_purchase  | Diff:     0.01 | 95% CI: [  -0.02,    0.04] | p-value = 0.558 
total_revenue        | Diff:    40.00 | 95% CI: [ -22.35,  101.04] | p-value = 0.508 
total_bonus          | Diff:    19.63 | 95% CI: [  16.57,   22.79] | p-value = 0.491 
additional_cost      | Diff:  1843.43 | 95% CI: [1693.94, 2000.70] | p-value = 0.486 

--- АКЦИЯ 6%_otk (размер групп после балансировки: 342) ---
avg_check            | Diff:    36.99 | 95% CI: [-108.13,  308.47] | p-value = 0.604 
transaction_count    | Diff:     0.04 | 95% CI: [  -0.06,    0.14] | p-value = 0.575 
has_repeat_purchase  | Diff:     0.02 | 95% CI: [  -0.04,    0.08] | p-value = 0.654 
total_revenue        | Diff:    21.02

## 📊 ИТОГОВЫЕ ВЫВОДЫ ПО АНАЛИЗУ ЭФФЕКТИВНОСТИ АКЦИЙ

### ⚠️ КЛЮЧЕВАЯ ПРОБЛЕМА:  ДИСБАЛАНС ГРУПП

*   **Акция 7% "Новые"**: 13,378 клиентов в тесте vs 1,474 в контроле (**дисбаланс 80.2%**).
*   **Акция 6% "Отток"**: 3,545 клиентов в тесте vs 342 в контроле (**дисбаланс 82.4%**).
*   **Отсутствие пересечений клиентов** между тестом и контролем внутри каждой акции подтверждает корректность разделения групп на уровне данных.

### 🧪 СТАТИСТИЧЕСКИЙ АНАЛИЗ (БУТСТРЭП С БАЛАНСИРОВКОЙ)

Для корректной оценки эффекта проведен анализ методом бутстрэп с балансировкой размеров выборок до минимального общего размера.

**Результаты для поведенческих метрик (p-value > 0.05, 0 звезд):**

| Акция      | Метрика               | Наблюдаемая разница | 95% ДИ                    | p-value | Стат. значимость |
| :--------- | :-------------------- | :------------------ | :------------------------ | :------ | :--------------- |
| **7% Новые** | Средний чек           | +41.05              | [-24.29, 106.65]          | 0.516   | ❌               |
|            | Кол-во транзакций     | +0.01               | [-0.03, 0.05]             | 0.587   | ❌               |
|            | Доля повторных покупок| +0.01 (1.1 п.п.)    | [-0.02, 0.04]             | 0.558   | ❌               |
|            | Выручка на клиента    | +40.00              | [-22.35, 101.04]          | 0.508   | ❌               |
| **6% Отток** | Средний чек           | +36.99              | [-108.13, 308.47]         | 0.604   | ❌               |
|            | Кол-во транзакций     | +0.04               | [-0.06, 0.14]             | 0.575   | ❌               |
|            | Доля повторных покупок| +0.02 (2.0 п.п.)    | [-0.04, 0.08]             | 0.654   | ❌               |
|            | Выручка на клиента    | +21.02              | [-126.97, 287.34]         | 0.765   | ❌               |

**Вывод:** По поведенческим метрикам статистически значимой разницы между тестовой и контрольной группами выявить **не удалось**.

### 🔋 АНАЛИЗ МОЩНОСТИ ТЕСТА

Из-за серьезного дисбаланса размеров групп мощность теста крайне низкая:

*   **Акция 7% "Новые" (n=1474):**
    *   Мощность для `avg_check`: **40.7%**
    *   Мощность для `transaction_count`: **15.6%**
    *   Мощность для `has_repeat_purchase`: **20.3%**
    *   Мощность для `total_revenue`: **41.8%**
*   **Акция 6% "Отток" (n=342):**
    *   Мощность для `avg_check`: **8.2%**
    *   Мощность для `transaction_count`: **17.8%**
    *   Мощность для `has_repeat_purchase`: **11.7%**
    *   Мощность для `total_revenue`: **6.0%**

**❗ КРИТИЧЕСКИ ВАЖНО:** Низкая мощность означает **высокий риск пропустить реальный эффект**. Даже если акция действительно работает, текущий дизайн теста не позволяет это обнаружить с высокой вероятностью.

### 💰 АНАЛИЗ МЕТРИК АКЦИИ (ОЖИДАЕМЫЕ РАЗЛИЧИЯ)

Различия в метриках `total_bonus` и `additional_cost` статистически также не значимы (p-value > 0.05), но доверительные интервалы не включают 0. Это ожидаемо, так как:
*   Эти метрики **напрямую зависят** от условий акции.
*   Акция гарантированно увеличивает бонусы (`total_bonus`) и расходы партнера (`additional_cost`) для первой транзакции.
*   Основная цель анализа - оценить **влияние на поведение клиентов**, а не на механику начисления бонусов.

### ✅ ОБЩИЙ ВЫВОД

На основании проведенного анализа **невозможно достоверно утверждать об эффективности** акций "7% на Новые" и "6% на Отток по поведенческим метрикам.

**Основная причина:** **Критически низкая мощность статистических тестов** из-за **серьезного дисбаланса размеров** тестовой и контрольной групп.

### 📌 РЕКОМЕНДАЦИИ

1.  **Для будущих акций:** Необходимо обеспечить **существенное увеличение размера контрольных групп** (ориентировочно до 30-50% от размера тестовой группы) для проведения надежного статистического анализа.
2.  **Для текущих данных:** Из-за низкой мощности теста **нельзя делать однозначных выводов** о наличии или отсутствии эффекта. Требуется повторный анализ на данных с корректно сформированными группами.
